In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from utils_lstm import load_lstm_dfs, load_train_test_lstm
from lstm_data_prep import LstmDataPrep
from sklearn.preprocessing import StandardScaler


2024-02-28 17:10:24.029529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
list_of_dfs=load_lstm_dfs()
train_set, test_set = load_train_test_lstm(list_of_dfs=list_of_dfs)

In [4]:
print("Train Shape: ",train_set.shape)
print("Test Shape:" ,test_set.shape)

Train Shape:  (13600, 10)
Test Shape: (37401, 10)


New Try

In [5]:
config = {
    'target_column': 'anomaly',
    'n_steps' : 5,
    'num_splits' : 5,
    }

In [6]:
model_pipe = LstmDataPrep(**config).run()

In [7]:
model_pipe

Pipeline(steps=[('transformations',
                 FeatureUnion(transformer_list=[('part_0',
                                                 Pipeline(steps=[('drop_targets',
                                                                  DropColumnTransformer(columns_to_drop=['changepoint',
                                                                                                         'anomaly'])),
                                                                 ('classifier',
                                                                  <lstm_model.LstmModel object at 0x14212a380>)])),
                                                ('part_1',
                                                 Pipeline(steps=[('drop_targets',
                                                                  DropColumnTransformer(columns_to_drop=['changepoint',
                                                                                                         'anomaly'])),
                                                                 ('classi...
                                                 Pipeline(steps=[('drop_targets',
                                                                  DropColumnTransformer(columns_to_drop=['changepoint',
                                                                                                         'anomaly'])),
                                                                 ('classifier',
                                                                  <lstm_model.LstmModel object at 0x14212a710>)])),
                                                ('part_4',
                                                 Pipeline(steps=[('drop_targets',
                                                                  DropColumnTransformer(columns_to_drop=['changepoint',
                                                                                                         'anomaly'])),
                                                                 ('classifier',
                                                                  <lstm_model.LstmModel object at 0x14212a860>)]))])),
                ('processing_results', AggregateModelScores())])

In [ ]:
res1 = model_pipe.fit_transform(train_set)

In [ ]:
res1

In [ ]:
res2 = [model_pipe.transform(df) for df in  list_of_dfs]

In [ ]:
res3 = pd.concat(res2, ignore_index=True)
res3

In [ ]:
y_pred = res3['avg_prediction']
y_pred.shape

In [ ]:
base_preds = pd.read_csv('../data/lstm_baseline_predictions.csv')['y_pred']

In [ ]:
len(base_preds)

In [ ]:
len(y_pred)

In [ ]:
y_test = pd.concat([df['anomaly'][5:] for df in list_of_dfs],ignore_index=True)
len(y_test)

In [ ]:
models_config = {
    'baseline' : {
        'y_test' :y_test.values,
        'y_pred': base_preds.values,
        'th': 0.5
    },
    
    'new_model' : {
        'y_test' :y_test.values,
        'y_pred':y_pred.values,
        'th' : 0.91
    }
}

In [ ]:
models_config

In [ ]:
from evaluation import ModelEvaluation

In [ ]:
eval = ModelEvaluation(eval_config=models_config)

In [ ]:
# eval.calc_metrics()

In [ ]:
eval.eval_config['new_model'].keys()

In [ ]:
eval.plot_metrics()

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import f1_score
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
# from sklearn.metrics import roc_curve, auc
# import numpy as np

In [ ]:
# def calc_metrics(y_true,y_pred):
#     th = 0.91
#     accuracy = accuracy_score(y_true, np.where(y_pred>=th,1,0))
#     cm = confusion_matrix(y_true, np.where(y_pred>=th,1,0))
#     f1 = f1_score(y_true, np.where(y_pred>=th,1,0))
#     fpr, tpr, thresholds = roc_curve(y_true, y_pred)
#     roc_auc = auc(fpr, tpr)
#     return accuracy,cm,f1,fpr, tpr, thresholds, roc_auc

# def plot_metrics(cm, fpr, tpr, roc_auc, thresholds, title = 'Model Evaluation Metrics'):
    
#     fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))
#     fig.suptitle(title, fontsize=16, y=1.02)
#     confusion_matrix_ax = ax[0]
#     roc_curve_ax = ax[1]

#     # Confusion Matrix in the first subplot
#     ConfusionMatrixDisplay(confusion_matrix=cm).plot(ax=confusion_matrix_ax,cmap='Blues')
#     confusion_matrix_ax.set_title('Confusion Matrix')
#     confusion_matrix_ax.set_xlabel('Predicted labels')
#     confusion_matrix_ax.set_ylabel('True labels')

#     # ROC Curve in the second subplot
#     roc_curve_ax.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
#     roc_curve_ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
#     roc_curve_ax.set_xlabel('False Positive Rate')
#     roc_curve_ax.set_ylabel('True Positive Rate')
#     roc_curve_ax.set_title('Receiver Operating Characteristic (ROC) Curve')
#     roc_curve_ax.legend(loc='lower right')
#     # Display the plot
#     plt.tight_layout()
#     plt.show()

In [ ]:
# accuracy, cm, f1, fpr, tpr, thresholds, roc_auc = calc_metrics(y_test,y_pred)
# plot_metrics(cm, fpr, tpr, roc_auc, thresholds, title = 'Average Probability')

## MAR

In [ ]:
2239/(2239+10828)

In [ ]:
from lstm_full_model import FullLstmPipeline

In [ ]:
model_config = {
    'target_column': 'anomaly',
    'n_steps' : 5,
    'num_splits' : 34,
    }

In [ ]:
full_model = FullLstmPipeline(model_config)

In [ ]:

preds = full_model.run()

In [ ]:
preds

In [ ]:
full_model.y_test

In [ ]:
model_config1 = {
    'target_column': 'anomaly',
    'n_steps' : 5,
    'num_splits' : 34,
    'th' : 0.91
    }

In [ ]:
from evaluation import ModelSelector

In [ ]:
selector = ModelSelector(model_config1,FullLstmPipeline,min_splits=32,max_splits=34)

In [ ]:
res = selector.run_model_permutations()

In [ ]:
res[0].style.background_gradient(axis=1)